## Hacker News Porject
In this project, we'll work with a data set of submissions to popular technology site [Hacker News](https://news.ycombinator.com/).


Hacker News is a site started by the startup incubator [Y Combinator](https://www.ycombinator.com/), where user-submitted stories (known as "posts") receive votes and comments, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of the Hacker News listings can get hundreds of thousands of visitors as a result.

You can find the data set here, but note that we have reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that didn't receive any comments and then randomly sampling from the remaining submissions. Below are descriptions of the columns:

- `id`: the unique identifier from Hacker News for the post
- `title`: the title of the post
- `url`: the URL that the posts links to, if the post has a URL
- `num_points`: the number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
- `num_comments`: the number of comments on the post
- `author`: the username of the person who submitted the post
- `created_at`: the date and time of the post's submission

We're specifically interested in posts with titles that begin with either `Ask HN` or `Show HN`. Users submit `Ask HN` posts to ask the Hacker News community a specific question. 

We'll compare these two types of posts to determine the following:

- Do `Ask HN` or `Show HN` receive more comments on average?
- Do posts created at a certain time receive more comments on average?

In [1]:
# Slide 1: Reading the CSV file and assigning to variable 

from csv import reader  # if we do not want to use pandas, we can use the "csv" library and import reader from it
opened = open ("hacker_news.csv")
hn = list (reader (opened)) #reading as a list of lists
print(hn[0:6])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


In [3]:
#Slide 2: Extracting the first row of data and assigning it to the header

headers = hn[0]
hn = hn[1:]

print(headers)
print("\n")
print(hn[0:6])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12'], ['10482257', '

Now that we've removed the headers from hn, we're ready to filter our data. Since we're only concerned with post titles beginning with Ask HN or Show HN, we'll create new lists of lists containing just the data for those titles.

To find the posts that begin with either Ask HN or Show HN, we'll use the string method `startswith`. Given a string object, say, string1, we can check if starts with, say, dq by inspecting the output of the object string1.startswith('dq'). If string1 starts with dq, it will return True; otherwise, it will return False.

In [5]:
#Slide 3: Using startswith string method to put the posts in appropriate bins.

ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    if title.lower().startswith("ask hn"):
        ask_posts.append(row)
    elif title.lower().startswith("show hn"):
        show_posts.append(row)
    else:
        other_posts.append(row)

print("ask posts = ",len(ask_posts))
print("show posts = ",len(show_posts))
print("other posts = ",len(other_posts))

ask posts =  1744
show posts =  1162
other posts =  17194


In [6]:
#Slide 4: Finding the total number of comments on the ask posts. 
total_ask_comments = 0

for row in ask_posts:
    total_ask_comments += int(row [4]) #num_comments is index 4

#calculating the avg ask comments
avg_ask_comments = total_ask_comments/len(ask_posts) 
print("average ask comments is:",avg_ask_comments)


total_show_comments = 0

for row in show_posts:
    total_show_comments += int(row [4]) #num_comments is index 4

#calculating the avg show comments
avg_show_comments = total_show_comments/len(show_posts) 
print("average show comments is:",avg_show_comments)

average ask comments is: 14.038417431192661
average show comments is: 10.31669535283993


The above investigation shows that on avergae the ask posts have 4 more comments than the show posts. The average ask post gets around 14 comments, where as the average show post gets only 10. 

In [8]:
#Slide 5: Finding the number of ask posts and comments by hour created using the 

import datetime as dt

result_list = []


for row in ask_posts:
    
    result_list.append(row[7:3:-2]) # Slicing the larger list to get the create time and comments 
    ###but I have not made the second element to int yet.
    ###I could go: which gives the int for the comment number
#     temp = []
#     temp.append(row[6])
#     temp.append(int(row[4]))
#     result_list.append(temp)
        
counts_by_hour = {}
comments_by_hour = {}

#print(result_list[0:3])

date_format = ("%m/%d/%Y %H:%M")

for row in result_list:
    date_time =  dt.datetime.strptime(row[0], date_format)
    time = dt.datetime.strftime(date_time, "%H")
    if time not in counts_by_hour:
        counts_by_hour[time] = 1
        comments_by_hour[time] = int(row[1])       
    else:
        counts_by_hour[time] += 1
        comments_by_hour[time] += int(row[1]) 


    
print("comment by hour ",comments_by_hour)
print('\n')
print("counts by hour",counts_by_hour)

# counts_by_hour: contains the number of ask posts created during each hour of the day.
# comments_by_hour: contains the corresponding number of comments ask posts created at each hour received.


comment by hour  {'09': 251, '13': 1253, '10': 793, '14': 1416, '16': 1814, '23': 543, '12': 687, '17': 1146, '15': 4477, '21': 1745, '20': 1722, '02': 1381, '18': 1439, '03': 421, '05': 464, '19': 1188, '01': 683, '22': 479, '08': 492, '04': 337, '00': 447, '06': 397, '07': 267, '11': 641}


counts by hour {'09': 45, '13': 85, '10': 59, '14': 107, '16': 108, '23': 68, '12': 73, '17': 100, '15': 116, '21': 109, '20': 80, '02': 58, '18': 109, '03': 54, '05': 46, '19': 110, '01': 60, '22': 71, '08': 48, '04': 47, '00': 55, '06': 44, '07': 34, '11': 58}


In [9]:
#Slide 6: Doing the avergae operation on all the vlaues in the dictionary and returning the result as a list of lists. 

avg_by_hour =[]
for hour in comments_by_hour:
    avg_by_hour.append([hour,comments_by_hour[hour]/counts_by_hour[hour]])

print("average comments per post by the hour is:",avg_by_hour[0:5])

average comments per post by the hour is: [['09', 5.5777777777777775], ['13', 14.741176470588234], ['10', 13.440677966101696], ['14', 13.233644859813085], ['16', 16.796296296296298]]


Although we now have the results we need, this format makes it difficult to identify the hours with the highest values. Let's finish by sorting the list of lists and printing the five highest values in a format that's easier to read.

In [11]:
#Slide 7: Sorting and finding out the highest values.

swap_avg_by_hour = []

for row in avg_by_hour:  #used the "temp" trick learnt form answers, but the slicing is a lot better.
    temp = []
    temp.append(row[1])
    temp.append(row[0])
    swap_avg_by_hour.append(temp)

sorted_swap = sorted(swap_avg_by_hour, reverse = True)

print("Top 5 Hours for Ask Posts Comments")
print(sorted_swap[0:6])
print("Other method:--")

# Different method, using string formatting.
for row in sorted_swap[0:5]:
    hour_prsd = dt.datetime.strptime(row[1],"%H")
    hour_frmtd = dt.datetime.strftime(hour_prsd,"%H:00")
    
    #print(hour_frmtd)
    

    my_str = "{}: {:.2f} average comments per post"
    output = my_str.format(hour_frmtd,row[0])
    print(output)
 
    

Top 5 Hours for Ask Posts Comments
[[38.5948275862069, '15'], [23.810344827586206, '02'], [21.525, '20'], [16.796296296296298, '16'], [16.009174311926607, '21'], [14.741176470588234, '13']]
Other method:--
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


**Conclusions:** Data set has US eastern time. I am in Dubai which is 8 hours AHEAD of US easternn time, so the highest post for me would be 23:00 (11pm) local dubai time. The next highest is 10 am dubai time. 